In [1]:
# kaggle titanic project
# 0404/2021

In [154]:
# imports
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense,Dropout,Conv1D,MaxPooling1D,Conv2D,MaxPooling2D,Flatten,LSTM,Embedding
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [508]:
# read in data
def read_data(train,test):
    train_data, test_data = pd.read_csv(train), pd.read_csv(test)
    train_data = train_data.drop(["PassengerId","Name"],axis=1)
    test_data = test_data.drop(["Name"],axis=1)
    return train_data, test_data

In [509]:
train, test = read_data("/Users/zijianzhao/Downloads/titanic/train.csv", "/Users/zijianzhao/Downloads/titanic/test.csv")

In [510]:
# transform catogrical features to numerical vectors
def to_num(col):
    le = LabelEncoder()
    le.fit(col)
    return le.transform(col)

In [511]:
# Convert catogrical features to numerical vectors
train["Sex"] = to_num(train["Sex"])
train["Ticket"] = to_num(train["Ticket"])
train["Cabin"] = to_num(train["Cabin"])

test["Sex"] = to_num(test["Sex"])
test["Ticket"] = to_num(test["Ticket"])
test["Cabin"] = to_num(test["Cabin"])

In [512]:
# Normalize numerical features with large interval
def norm(col):
    col = col.values.reshape(-1,1)
    scaler = MinMaxScaler()
    scaler.fit(col)
    return scaler.transform(col)

In [513]:
train["Age"] = norm(train["Age"])
train["Fare"] = norm(train["Fare"])
train["Ticket"] = norm(train["Ticket"])
train["Cabin"] = norm(train["Cabin"])

test["Age"] = norm(test["Age"])
test["Fare"] = norm(test["Fare"])
test["Ticket"] = norm(test["Ticket"])
test["Cabin"] = norm(test["Cabin"])


In [514]:
train_data = pd.get_dummies(train, columns = ["Pclass", "Embarked","SibSp"])
test_data = pd.get_dummies(test, columns = ["Pclass", "Embarked","SibSp"])


In [515]:
train_data, validate_data = train_test_split(train_data,test_size=0.15)
train_Y, train_X = train_data["Survived"], train_data.drop("Survived",axis=1)
validate_Y, validate_X = validate_data["Survived"], validate_data.drop("Survived",axis=1)
train = xgb.DMatrix(train_X, label=train_Y, enable_categorical=True)
validate = xgb.DMatrix(validate_X, label=validate_Y, enable_categorical=True)
test_id = test_data["PassengerId"]
test = test_data.drop("PassengerId",axis=1)
test = xgb.DMatrix(test, enable_categorical=True)

In [ ]:
test_id

In [517]:
import xgboost as xgb

In [532]:
param = {
    'max_depth': 10,
    'eta': 0.3,
    'objective': 'multi:softmax',
    'num_class': 2,
    #'eval_metric': 'merror'
}
epochs = 10

In [533]:
model = xgb.train(param, train, epochs)

[23:12:00] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [534]:
predictions = model.predict(validate)

In [535]:
predictions

array([0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
      dtype=float32)

In [536]:
from sklearn.metrics import accuracy_score

In [537]:
accuracy_score(validate_Y,predictions)

0.8432835820895522

In [528]:
sub = model.predict(test)

In [529]:
sub = pd.DataFrame(sub,columns=["Survived"])
sub = pd.concat([pd.DataFrame(test_id),sub.astype(int)],axis=1)

In [530]:
sub.to_csv("predictions.csv",index=None)

In [531]:
sub

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
